In [1]:
import collections
import math
import os

import docker
import fire
import torch

from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, AutoModelForCausalLM
import time
import evaluate
import pandas as pd
import numpy as np

import peft
from peft import LoraConfig, get_peft_model, TaskType, PeftModel, PeftConfig

import transformers
from datasets import Dataset

from transformers import T5Config
from transformers import T5ForConditionalGeneration



from fedn.utils.helpers.helpers import get_helper, save_metadata, save_metrics 


In [2]:
HELPER_MODULE = 'numpyhelper'
helper = get_helper(HELPER_MODULE)

In [3]:
def load_parameters(model_path):
    """ Load model parameters from file and populate model.

    param model_path: The path to load from.
    :type model_path: str
    :return: The loaded model.
    :rtype: torch.nn.Module
    """
    parameters_np = helper.load(model_path)
    model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
    params_dict = zip(model.state_dict().keys(), parameters_np)
    state_dict = collections.OrderedDict({key: torch.tensor(x) for key, x in params_dict})
    model.load_state_dict(state_dict, strict=True)
    
    return model

In [4]:
model_name='google/flan-t5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
in_model_path = '/Users/nash/Downloads/Models/2c_4e_10/0f09c020-c528-4e0b-98c9-96c16b08e079'

In [6]:
peft_model = load_parameters(in_model_path)

In [7]:
dataset = pd.read_csv("/Users/nash/Project/fedn/fedn/examples/mnist-pytorch/rouge_dataset.csv")
dialogues = dataset['dialogue'][0:50]
human_baseline_summaries = dataset['summary'][0:50]

    
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
        prompt = f"""
    Summarize the following conversation.

    {dialogue}

    Summary: """
        
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids

        peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
        peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

        
        peft_model_summaries.append(peft_model_text_output)

     
    # Evaluate
rouge = evaluate.load('rouge')

peft_model_results = rouge.compute(
        predictions=peft_model_summaries,
        references=human_baseline_summaries[0:len(peft_model_summaries)],
        use_aggregator=True,
        use_stemmer=True,
)
    # JSON schema
report = {
        "rouge1": peft_model_results["rouge1"],
        "rouge2": peft_model_results["rouge2"],
        "rougeL": peft_model_results["rougeL"],
        "rougeLsum": peft_model_results["rougeLsum"],
    }

In [8]:
report

{'rouge1': 0.1789229999834666,
 'rouge2': 0.04065469943744764,
 'rougeL': 0.12541153850188136,
 'rougeLsum': 0.1256964182458587}

In [22]:
device = torch.device("mps" if torch.cuda.is_available() else "cpu")

In [23]:
device

device(type='cpu')